# Modélisation et Évaluation du Marketing Mix Model

Ce notebook présente l'entraînement, l'évaluation et l'interprétation du modèle MMM pour optimiser les allocations budgétaires marketing.

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
import os
import sys
import json

# Configuration de Matplotlib
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

# Ajouter le répertoire parent au chemin Python
sys.path.append('..')

In [ ]:
# Initialiser une session Spark
spark = SparkSession.builder \
    .appName("mmm_modeling") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

## 1. Chargement des données prétraitées

In [ ]:
# Charger la configuration
with open("../config/online_retail_config.json", "r") as f:
    config = json.load(f)

# Charger les données d'entraînement et de test
train_df = spark.read.parquet("../data/train_data.parquet")
test_df = spark.read.parquet("../data/test_data.parquet")

print(f"Ensemble d'entraînement: {train_df.count()} lignes, {len(train_df.columns)} colonnes")
print(f"Ensemble de test: {test_df.count()} lignes, {len(test_df.columns)} colonnes")

# Afficher un aperçu
print("\nAperçu des données d'entraînement:")
train_df.select(["date", "revenue"] + config["marketing_channels"]).show(5)

## 2. Entraînement du modèle MMM

In [ ]:
from src.models.mmm_model import MMMModel

# Initialiser le modèle MMM
mmm_model = MMMModel(spark, "../config/online_retail_config.json")

# Entraîner le modèle
print("Entraînement du modèle MMM...")
model, feature_importances = mmm_model.train_model(train_df)

# Afficher les importances des caractéristiques
print("\nTop 20 des caractéristiques les plus importantes:")
print(feature_importances.head(20))

# Visualiser les importances
plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=feature_importances.head(20))
plt.title('Top 20 des caractéristiques les plus importantes')
plt.xlabel('Importance')
plt.ylabel('Caractéristique')
plt.tight_layout()
plt.show()

## 3. Évaluation du modèle

In [ ]:
# Évaluer le modèle
print("Évaluation du modèle sur l'ensemble de test...")
metrics = mmm_model.evaluate_model(model, test_df)

# Afficher les métriques
print("\nMétriques d'évaluation:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Préparer les données pour visualiser les prédictions
X_test, y_test = mmm_model.prepare_training_data(test_df)
y_pred = model.predict(X_test)

# Tracer les prédictions vs réalité
test_dates = test_df.select("date").toPandas()["date"]
result_df = pd.DataFrame({
    "date": test_dates,
    "actual": y_test,
    "predicted": y_pred
})
result_df = result_df.sort_values("date")

plt.figure(figsize=(14, 7))
plt.plot(result_df["date"], result_df["actual"], 'b-', label='Ventes réelles')
plt.plot(result_df["date"], result_df["predicted"], 'r-', label='Ventes prédites')
plt.title('Ventes réelles vs. prédites sur l\'ensemble de test')
plt.xlabel('Date')
plt.ylabel('Ventes (£)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tracer un diagramme de dispersion
plt.figure(figsize=(10, 8))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Ventes réelles (£)')
plt.ylabel('Ventes prédites (£)')
plt.title('Diagramme de dispersion: ventes réelles vs. prédites')
plt.grid(True)
plt.tight_layout()
plt.show()

## 4. Analyse des contributions des canaux

In [ ]:
# Calculer les contributions des canaux
print("Calcul des contributions des canaux...")
full_df = spark.read.parquet("../data/mmm_features.parquet")
contributions_df = mmm_model.calculate_channel_contributions(model, full_df)

# Sauvegarder les contributions pour une utilisation ultérieure
contributions_df.to_csv("../reports/channel_contributions.csv", index=False)
print("Contributions sauvegardées dans ../reports/channel_contributions.csv")

# Calculer les contributions moyennes
channel_contribs = {}
channel_contribs['baseline'] = contributions_df['baseline_contribution'].mean()

for channel in config['marketing_channels']:
    contrib_col = f"{channel}_contribution"
    if contrib_col in contributions_df.columns:
        channel_contribs[channel] = contributions_df[contrib_col].mean()

# Créer un DataFrame pour le graphique
contrib_df = pd.DataFrame({
    'channel': list(channel_contribs.keys()),
    'contribution': list(channel_contribs.values()),
    'contribution_pct': [v / contributions_df['predicted_revenue'].mean() * 100 for v in channel_contribs.values()]
}).sort_values('contribution', ascending=False)

# Afficher les contributions
print("\nContributions moyennes par canal:")
print(contrib_df)

# Visualiser les contributions
plt.figure(figsize=(12, 6))
sns.barplot(x='channel', y='contribution', data=contrib_df)
plt.title('Contribution moyenne par canal marketing')
plt.xlabel('Canal')
plt.ylabel('Contribution aux ventes (£)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Graphique en camembert des pourcentages
plt.figure(figsize=(10, 10))
plt.pie(contrib_df['contribution_pct'], labels=contrib_df['channel'], autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Répartition des contributions (%)')
plt.tight_layout()
plt.show()

## 5. Analyse du ROI par canal

In [ ]:
# Calculer le ROI médian par canal
roi_data = {}
for channel in config['marketing_channels']:
    roi_col = f"{channel}_roi"
    if roi_col in contributions_df.columns:
        roi_data[channel] = contributions_df[roi_col].median()

# Créer un DataFrame pour le graphique
roi_df = pd.DataFrame({
    'channel': list(roi_data.keys()),
    'roi': list(roi_data.values())
}).sort_values('roi', ascending=False)

# Afficher le ROI
print("\nROI médian par canal:")
print(roi_df)

# Visualiser le ROI
plt.figure(figsize=(12, 6))
sns.barplot(x='channel', y='roi', data=roi_df)
plt.title('ROI médian par canal marketing')
plt.xlabel('Canal')
plt.ylabel('ROI (£ générés par £ dépensée)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Optimisation de l'allocation budgétaire

In [ ]:
# Optimiser l'allocation budgétaire
print("Optimisation de l'allocation budgétaire...")
budget_allocation = mmm_model.optimize_budget(contributions_df)

# Sauvegarder l'allocation
budget_allocation.to_csv("../reports/budget_allocation.csv", index=False)
print("Allocation budgétaire sauvegardée dans ../reports/budget_allocation.csv")

# Afficher l'allocation
print("\nAllocation budgétaire optimisée:")
print(budget_allocation)

# Visualiser l'allocation
plt.figure(figsize=(12, 7))

# Créer un graphique avec deux axes Y
fig, ax1 = plt.subplots(figsize=(12, 7))

# Premier axe pour le budget
bars = ax1.bar(budget_allocation['channel'], budget_allocation['budget'], color='skyblue')
ax1.set_xlabel('Canal')
ax1.set_ylabel('Budget alloué (£)', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')

# Ajouter les valeurs sur les barres
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 1000,
            f'£{height:.0f}', ha='center', va='bottom', fontsize=10)

# Deuxième axe pour le ROI
ax2 = ax1.twinx()
ax2.plot(budget_allocation['channel'], budget_allocation['roi'], 'ro-', linewidth=2, markersize=8)
ax2.set_ylabel('ROI', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.title('Allocation budgétaire optimisée et ROI par canal')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Analyse des contributions au fil du temps

In [ ]:
# Analyser l'évolution des contributions dans le temps
print("Analyse des contributions au fil du temps...")

# Convertir la colonne date en datetime
contributions_df['date'] = pd.to_datetime(contributions_df['date'])
contributions_df = contributions_df.sort_values('date')

# Créer un graphique des contributions au fil du temps
plt.figure(figsize=(14, 8))

# Tracer la contribution de base
plt.plot(contributions_df['date'], contributions_df['baseline_contribution'], 
         label='Baseline', linewidth=2, color='gray')

# Tracer les contributions par canal
for channel in config['marketing_channels']:
    contrib_col = f"{channel}_contribution"
    if contrib_col in contributions_df.columns:
        plt.plot(contributions_df['date'], contributions_df[contrib_col], 
                 label=channel, linewidth=2)

plt.title('Évolution des contributions par canal au fil du temps')
plt.xlabel('Date')
plt.ylabel('Contribution (£)')
plt.grid(True)
plt.legend(loc='best')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Créer un graphique empilé pour montrer la composition totale
plt.figure(figsize=(14, 8))

# Préparer les données
channels = ['baseline'] + config['marketing_channels']
contrib_cols = [f"{ch}_contribution" if ch != 'baseline' else 'baseline_contribution' 
               for ch in channels if f"{ch}_contribution" in contributions_df.columns 
               or ch == 'baseline']
    
# Créer le graphique empilé
plt.stackplot(contributions_df['date'], 
              [contributions_df[col] for col in contrib_cols],
              labels=[col.replace('_contribution', '') for col in contrib_cols])

plt.title('Composition des ventes au fil du temps')
plt.xlabel('Date')
plt.ylabel('Ventes (£)')
plt.grid(True)
plt.legend(loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Génération d'un rapport complet

In [ ]:
from src.visualization.visualization import MMMVisualization

# Initialiser la classe de visualisation
visualizer = MMMVisualization("../config/online_retail_config.json")

# Sauvegarder les métriques
with open("../reports/model_metrics.json", "w") as f:
    json.dump(metrics, f)

# Générer le rapport
print("Génération du rapport...")
report_path = visualizer.generate_report(
    metrics, 
    contributions_df, 
    feature_importances, 
    budget_allocation
)

print(f"Rapport généré: {report_path}")
print("\nAnalyse MMM terminée ! Visualisez les résultats dans le dashboard Streamlit.")

In [ ]:
# Arrêter la session Spark
spark.stop()